In [59]:
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import requests
import os
from functools import reduce
from itertools import chain
import matplotlib.pyplot as plt

In [ ]:
import kaggle
import os

os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""

!kaggle datasets download -d skylarkphantom/mit-datacenter-challenge-data

# Unzip the file
with ZipFile("mit-datacenter-challenge-data.zip", 'r') as zip_ref:
    zip_ref.extractall("mit_data")

In [ ]:
!kaggle datasets download -d skylarkphantom/mit-datacenter-challenge-data

In [ ]:
# Unzip the file
with ZipFile("mit-datacenter-challenge-data.zip", 'r') as zip_ref:
    zip_ref.extractall("mit_data")

# Load the specific CSV file


In [64]:
df = pd.read_csv("mit_data/scheduler_data.csv")
df.columns

Index(['id_array_job', 'id_array_task', 'id_user', 'kill_requid',
       'nodes_alloc', 'nodelist', 'cpus_req', 'derived_ec', 'exit_code',
       'gres_req', 'gres_alloc', 'gres_used', 'array_max_tasks',
       'array_task_pending', 'constraints', 'flags', 'mem_req', 'partition',
       'priority', 'state', 'timelimit', 'time_submit', 'time_eligible',
       'time_start', 'time_end', 'time_suspended', 'track_steps', 'tres_alloc',
       'tres_req', 'job_type', 'id_job'],
      dtype='object')

In [9]:
len(df['id_user']) - len(df['id_user'].unique())

286885

In [57]:
len(df['time_start']) - len(df['time_start'].unique())

191073

In [58]:
len(df['time_start'])

287173

In [10]:
# Get the user with the highest occurrence
most_frequent_user = df['id_user'].value_counts().idxmax()
most_frequent_user_count = df['id_user'].value_counts().max()

print(f"User ID with the most appearances: {most_frequent_user}")
print(f"Number of appearances: {most_frequent_user_count}")

User ID with the most appearances: 41415979807
Number of appearances: 165386


In [24]:
df['nodelist'] = df['nodelist'].str.slice(1, -1)
df['nodelist'] = df['nodelist'].str.split(',')

In [ ]:
unique_nodes = set(chain.from_iterable(df['nodelist']))

In [37]:
nodes = list(chain.from_iterable(df['nodelist']))

In [38]:
len(nodes)

494531

In [14]:
len(nodes)

287173

In [12]:
# Identify the most frequent user
most_frequent_user = df['id_user'].value_counts().idxmax()

# Filter the DataFrame to rows where this user appears
user_data = df[df['id_user'] == most_frequent_user]

# Display a few rows of data for this user
user_data.head()

,id_array_job,id_array_task,id_user,kill_requid,nodes_alloc,nodelist,cpus_req,derived_ec,exit_code,gres_req,...,time_submit,time_eligible,time_start,time_end,time_suspended,track_steps,tres_alloc,tres_req,job_type,id_job
9542,41161693674,4595979483,41415979807,51671871839,1,['r1900814-n172107'],1,0,0,gpu:2,...,7802094,7802094,7802094,7802103,-1,0,"1=1,2=9600,4=1,5=1,1001=2","1=1,2=9600,4=1,5=1,1001=2",OTHER,29849133170
9543,41161693674,4595979483,41415979807,51671871839,1,['r6959555-n172107'],1,0,0,gpu:2,...,7802128,7802128,7802128,7802329,-1,0,"1=1,2=9600,4=1,5=1,1001=2","1=1,2=9600,4=1,5=1,1001=2",OTHER,8328006237
9544,41161693674,4595979483,41415979807,51671871839,1,['r1900814-n172107'],1,0,0,gpu:2,...,7802136,7802136,7802136,7802328,-1,0,"1=1,2=9600,4=1,5=1,1001=2","1=1,2=9600,4=1,5=1,1001=2",OTHER,46700920486
9545,41161693674,4595979483,41415979807,51671871839,1,['r204106-n172107'],1,0,33280,gpu:2,...,7802137,7802137,7802137,7802328,-1,0,"1=1,2=9600,4=1,5=1,1001=2","1=1,2=9600,4=1,5=1,1001=2",OTHER,9757122442
9547,41161693674,4595979483,41415979807,51671871839,1,['r1051355-n172107'],1,0,0,gpu:2,...,7802139,7802139,7802139,7802331,-1,0,"1=1,2=9600,4=1,5=1,1001=2","1=1,2=9600,4=1,5=1,1001=2",OTHER,38076546964


In [ ]:
# Set time_start as the index
df['time_submit'] = pd.to_datetime(user_data['time_submit'], unit='s')
df['time_start'] = pd.to_datetime(user_data['time_start'], unit='s')
df['time_end'] = pd.to_datetime(user_data['time_end'], unit='s')
df['duration'] = df['time_end'] - df['time_start']
df.reset_index('time_start', inplace=True)

In [73]:
time_stamps = df.index

In [74]:
time_stamps

DatetimeIndex([                'NaT',                 'NaT',
                               'NaT', '1970-04-01 14:09:48',
               '1970-04-01 14:09:53', '1970-04-01 14:10:21',
               '1969-12-31 23:59:59', '1969-12-31 23:59:59',
               '1970-04-01 14:16:03', '1970-04-01 14:16:03',
               ...
                               'NaT',                 'NaT',
                               'NaT',                 'NaT',
                               'NaT',                 'NaT',
                               'NaT',                 'NaT',
                               'NaT',                 'NaT'],
              dtype='datetime64[ns]', name='time_start', length=287173, freq=None)

In [ ]:
# Resample to one-second frequency and sum the durations (or count the jobs)
time_series_df = df.resample('s').agg(total_duration=('duration', 'sum')).fillna(pd.Timedelta(0))

# Convert total_duration to seconds for plotting
time_series_df['total_duration_seconds'] = time_series_df['total_duration'].dt.total_seconds()

# Plotting the total workload every second
plt.figure(figsize=(12, 6))
plt.plot(time_series_df.index, time_series_df['total_duration_seconds'], marker='o')
plt.title('Total Workload Over Time (every second)')
plt.xlabel('Time')
plt.ylabel('Total Duration (seconds)')
plt.grid()
plt.show()

In [61]:
time_series_df['total_duration'].head()

time_start
1969-12-31 23:00:00    641297539634
1970-01-01 00:00:00               3
1970-01-01 01:00:00               1
1970-01-01 02:00:00               0
1970-01-01 03:00:00               0
Freq: h, Name: total_duration, dtype: int64

In [42]:
user_data['time_submit'] = pd.to_datetime(user_data['time_submit'], unit='s')
user_data['time_start'] = pd.to_datetime(user_data['time_start'], unit='s')
user_data['time_end'] = pd.to_datetime(user_data['time_end'], unit='s')
user_data['duration'] = df['time_end'] - df['time_start']

In [48]:
user_data[['time_start', 'duration']].tail(5)

,time_start,duration
275934,1970-12-16 11:21:00,43226
276172,1970-12-18 16:28:50,517032
277186,1970-12-22 10:18:37,43228
283121,1970-12-30 10:54:30,185912
284786,1971-01-02 22:01:29,208533


In [49]:
df['time_start'] = pd.to_datetime(df['time_start'], unit='s')

In [56]:
df['time_start'].sort_values().head()

152758   1969-12-31 23:59:59
46159    1969-12-31 23:59:59
46160    1969-12-31 23:59:59
46161    1969-12-31 23:59:59
104683   1969-12-31 23:59:59
Name: time_start, dtype: datetime64[ns]